In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

## Preprocess data

In [ ]:
!wget https://github.com/amanlai/datasets/raw/main/store-sales-time-series-forecasting.zip

--2024-02-22 09:35:49--  https://github.com/amanlai/datasets/raw/main/store-sales-time-series-forecasting.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/amanlai/datasets/main/store-sales-time-series-forecasting.zip [following]
--2024-02-22 09:35:49--  https://raw.githubusercontent.com/amanlai/datasets/main/store-sales-time-series-forecasting.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22416355 (21M) [application/zip]
Saving to: ‘store-sales-time-series-forecasting.zip’

store-sales-time-se 100%[===================>]  21.38M  --.-KB/s    in 0.09s   

2024-02-22 09:35:50 (229 MB/s) - 

In [ ]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.3 MB/s eta 0:00:00
time: 263 µs (started: 2024-02-22 09:35:59 +00:00)


In [ ]:
def read_data(filename, path='/content/store-sales-time-series-forecasting.zip', parse_dates=True, **kwargs):
    with zipfile.ZipFile(path) as zip_file:
        with zip_file.open(filename) as f:
            dates = ['date'] if parse_dates else None
            return pd.read_csv(f, parse_dates=dates, **kwargs)



def reshape_df(df, index='date', columns='store_nbr', values=None, end=None):

    p = df.pivot(index='date', columns='store_nbr', values=values)
    if end is not None:
        p = p[p.index <= end]
    p = p.interpolate(method='linear', limit_direction='both')
    return p



def transform_df(df, columns=None, log_transform=False, is_train=False):

    index = df.index

    if log_transform:
        df = np.log1p(df)

    sc = MinMaxScaler()
    scaled = sc.fit_transform(df)
    transformed = pd.DataFrame(scaled, index=index, columns=columns)

    if is_train:
        return transformed, sc
    else:
        return transformed



def get_target_data(df, target_cols='sales', train_end=None):

    p = reshape_df(df, 'date', 'store_nbr', target_cols, train_end)
    columns = p.columns.rename('store_nbr')
    result, sc = transform_df(p, columns, True, True)

    # create the static variables
    xy = df[['store_nbr', 'city', 'state', 'type', 'cluster']].drop_duplicates()
    dummies = pd.get_dummies(xy, columns=xy.columns).astype('int32')
    cols = dummies.columns.tolist()
    dummies = dummies.sort_values(cols, ascending=False, ignore_index=True)
    # since `dummies` is sorted, it's safe to assign `store_nbr` as its values
    dummies.index = result.columns
    return result, sc, dummies



def get_past_covariates(df, past_cols=None, train_end=None):

    p = reshape_df(df, 'date', 'store_nbr', past_cols, train_end)
    columns = pd.MultiIndex.from_tuples(p.columns, names=['past', 'store_nbr'])
    result = transform_df(p, columns)
    return result



def get_future_covariates(
    df,
    future_cols=None,
    future_ma_cols=None,
    future_window_sizes=None
):

    p = reshape_df(df, 'date', 'store_nbr', future_cols)
    columns = pd.MultiIndex.from_tuples(p.columns, names=['future', 'store_nbr'])
    transformed = transform_df(p, columns)
    ma_dfs = []
    for window_size in future_window_sizes:
        cols = pd.MultiIndex.from_product([
            [f'{c}_ma{window_size}' for c in future_ma_cols],
            transformed.columns.levels[1]
        ], names=['future', 'store_nbr'])
        ma = transformed[future_ma_cols].rolling(f'{window_size}D', center=True, closed='left').mean()
        ma.columns = cols
        ma_dfs.append(ma)
    result = pd.concat([transformed, *ma_dfs], axis=1)
    return result


def get_training_data(train_df, past_covariates, future_covariates, dummy_vars):

    # get column label order
    past_cols = past_covariates.columns.get_level_values('past').drop_duplicates()
    future_cols = future_covariates.columns.get_level_values('future').drop_duplicates()

    # prepare groupby objects to shift
    stacked_target = train_df.stack()
    target_gb = stacked_target.groupby(level='store_nbr')
    past_covs_gb = (
        past_covariates
        .stack()
        [past_cols]                        # <--- necessary for colab
        .groupby(level='store_nbr')
    )
    future_covs_gb = (
        future_covariates
        .stack(level='store_nbr')
        [future_cols]                      # <--- necessary for colab
        .groupby(level='store_nbr')
    )

    # shift the variables for appropriate number of time delta
    tmp = [
        *(target_gb.shift(i) for i in lags['target']),
        *(past_covs_gb.shift(i) for i in lags['past']),
        *(future_covs_gb.shift(i) for i in lags['future']),
    ]

    X_train = (
        pd.concat(tmp, axis=1)                          # concatenate the variables horizontally
        .dropna()                                       # drop NaNs created by shift
        .reset_index()                                  # recover `date`, `store_nbr` as columns
        .merge(dummy_vars, on='store_nbr', how='left')  # merge the static variables on `store_nbr`
        .set_index(['date', 'store_nbr'])               # make `date`, `store_nbr` back into index
        .sort_index(level='store_nbr')                  # sort by the store_nbr
    )
    y_train = stacked_target.loc[X_train.index]         # align y_train with X_train

    return X_train, y_train


def predict(
    model,
    n,
    train_df,
    past_covariates,
    future_covariates,
    dummy_vars,
    scaler,
    zero_forecast,
):

    relative_cov_lags = {
        'past': (np.array(lags['past']) - min(lags['past']))[::-1],
        'future': (np.array(lags['future']) - min(lags['future']))[::-1],
    }

    idx = train_df.columns
    target_array = train_df.tail(max(lags['target'])).values    # use the underlying array

    past_covs_array = (
        past_covariates
        .tail(max(lags['past']))
        # we can safely drop the outer level because past covariates consist of only one variable: transactions.
        .droplevel(0, axis=1)
        [idx]                     # sort so that the store numbers match
        .values                   # use the underlying array
    )

    dummy_vars = dummy_vars.loc[idx]   # sort so that the store numbers match

    cutoff = train_df.index.max() - pd.Timedelta(days=max(lags['future'])-1)
    cov_df = future_covariates.loc[cutoff:]

    # prediction
    predictions = []
    # t_pred indicates the number of time steps after the first prediction
    for t_pred in range(n):
        # concatenate the previous day's predictions to the target
        if predictions:
            last_prediction = predictions[-1][None, :]
            target_array = np.concatenate((target_array, last_prediction), axis=0)
        # prepare the target variable for prediction
        tmp_X = target_array[-np.array(lags['target'])]
        # prepare past covariates for prediction
        past_cov = past_covs_array[relative_cov_lags["past"] + t_pred]
        # prepare future covariates for prediction
        future_cov = (
            cov_df
            .iloc[relative_cov_lags['future'] + t_pred]  # select relevant dates for prediction
            .stack(level=0)                              # convert the outer column level (variable types) into index level
        )
        # the next 4 lines of code is to sort the inner level of future_cov in a specific way
        # useful for rapids; however, for pandas API,
        # we could pass `stack(level=0, sort=False)` instead of all this
        variables = future_covariates.columns.get_level_values(0).drop_duplicates()
        dates = future_cov.index.get_level_values(0).drop_duplicates()
        sorted_idx = pd.MultiIndex.from_product((dates, variables))
        future_cov = future_cov.loc[sorted_idx].values    # sort and use the underlying array

        # concatenate target, past and future covariates to pass to predict()
        # finally, we need to transpose this array because the model expects stores as index
        X = np.concatenate([tmp_X, past_cov, future_cov, dummy_vars.values.T], axis=0).T

        # prediction for a specific day
        predictions.append(model.predict(X))

    # invert the transformation
    pred_array = np.vstack(predictions)
    inv_trans_preds = np.expm1(scaler.inverse_transform(pred_array))
    predictions = pd.DataFrame(
        inv_trans_preds,
        index=train_df.index.max() + pd.timedelta_range(start='1D', periods=n, freq='D'),
        columns=idx
    )

    # if the past `zero_forecast` days were 0, predict 0
    zero_mask = train_df.tail(zero_forecast).sum() == 0
    predictions.loc[:, zero_mask] = 0
    # coerce negative predictions to be 0
    predictions = predictions.clip(0)
    return predictions

time: 2.33 ms (started: 2024-02-22 09:36:01 +00:00)


In [ ]:
train = read_data("train.csv")
test = read_data("test.csv")

oil = read_data('oil.csv').rename(columns={"dcoilwtico": "oil"})
store = read_data("stores.csv", parse_dates=False)
transactions = read_data("transactions.csv")
holiday_events = read_data("holidays_events.csv")

time: 2.56 s (started: 2024-02-22 09:36:02 +00:00)


In [ ]:
holiday_events['description'] = (
    holiday_events
    [['description', 'locale_name']]
    .apply(lambda x: x['description'].replace(x['locale_name'], ''), axis=1)
    .str.lower()
    .str.replace(r'\-|\+|\d+|\b(de|del|traslado|recupero|puente|-)\b', '', regex=True)
    .pipe(lambda x: x.str.extract('(futbol)', expand=False).fillna(x))
    .replace(np.unique(np.hstack(store[['city', 'state']].apply(lambda x: x.str.lower().unique()))), '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)
holiday_events = holiday_events[~holiday_events['transferred']]

time: 47 ms (started: 2024-02-22 09:36:05 +00:00)


Saturdays are designated as work days.

In [ ]:
work_days = (
    holiday_events
    .loc[holiday_events['type'] == 'Work Day', ["date", "type"]]
    .rename(columns={"type": "work_day"})
    .reset_index(drop=True)
)
work_days['work_day'] = work_days['work_day'].notna().astype('int32')

# remove work days after extracting above
holiday_events = holiday_events[holiday_events['type'] != 'Work Day'].reset_index(drop=True)

time: 12.1 ms (started: 2024-02-22 09:36:06 +00:00)


In [ ]:
relevant_days = (
    holiday_events
    .loc[holiday_events['locale'] == 'National', ["date", "description"]]
    .drop_duplicates()
)

dummified = pd.get_dummies(relevant_days, columns=['description'], prefix='holiday')

# some dates have multiple holidays, so put them all in a single row
national_holidays = dummified.groupby('date', as_index=False).sum()

# not all holidays are impactful
# keep some national holidays with larger impacts on sales
relevant_holidays = [
    'holiday_dia difuntos', 'holiday_dia la madre',
    'holiday_dia trabajo', 'holiday_futbol', 'holiday_navidad',
    'holiday_primer dia ano', 'holiday_terremoto'
]
national_holidays = national_holidays[['date', *relevant_holidays]]

time: 25.6 ms (started: 2024-02-22 09:36:07 +00:00)


In [ ]:
train_start, train_end = train['date'].agg(['min', 'max'])
test_start, test_end = test['date'].agg(['min', 'max'])

time: 29.7 ms (started: 2024-02-22 09:36:07 +00:00)


In [ ]:
# reindex training data
multi_idx = pd.MultiIndex.from_product(
    [pd.date_range(train_start, train_end), train['store_nbr'].unique(), train['family'].unique()],
    names=['date', 'store_nbr', 'family'],
)
# this adds missing dates
# however this generates missing sales, id and on-promotion values
train = train.set_index(['date', 'store_nbr', 'family']).reindex(multi_idx).reset_index()

# fill missing values with zeros
# the assumption here is that days where there were no sales were probably not recorded
train[['sales', 'onpromotion']] = train[['sales', 'onpromotion']].fillna(0)
# interpolate linearly as a filler for the 'id'
# not really useful for training but is useful for merging all datasets
train['id'] = train['id'].interpolate(method="linear")

time: 2.51 s (started: 2024-02-22 09:36:08 +00:00)


In [ ]:
# compute total sales for each store
store_sales = train.groupby(['date', 'store_nbr'], as_index=False)['sales'].sum()

# reindex transactions data
# same as the reindexing of `train`, this adds more dates and missing transactions
transactions = (
    transactions
    .merge(store_sales, on=['date', 'store_nbr'], how='outer')
    .sort_values(['date', 'store_nbr'], ignore_index=True)     # important for interpolation later on
)

# if there were zero sales, it is a good assumption that there would be zero transactions
transactions.loc[transactions['sales'] == 0, 'transactions'] = 0
transactions = transactions.drop(columns=['sales'])


# fill remaining missing values using linear interpolation
transactions['transactions'] = (
    transactions
    .groupby('store_nbr', group_keys=False)['transactions']
    .apply(lambda x: x.interpolate(method='linear', limit_direction='both'))
)

time: 392 ms (started: 2024-02-22 09:36:11 +00:00)


In [ ]:
# oil data is business day time series (therefore missing weekends)
# add those dates in by reindexing date

# date index of the date range
idx = pd.date_range(train_start, test_end, name='date')
# add the missing dates
oil = oil.set_index('date').reindex(idx).reset_index()
# interpolate the missing prices
oil['oil'] = oil['oil'].interpolate(method='linear', limit_direction='both')

time: 4.58 ms (started: 2024-02-22 09:36:11 +00:00)


In [ ]:
# combine all the datasets
data = (
    pd.concat([train, test])
    .merge(transactions, on=['date', 'store_nbr'], how='left')
    .merge(oil, on='date', how='left')
    .merge(store, on='store_nbr', how='left')
    .merge(work_days, on='date', how='left')
    .merge(national_holidays, on='date', how='left')
    .sort_values(['date', 'store_nbr', 'family'], ignore_index=True)
)

# fill columns with 0s to indicate absence of holidays/events
data[["work_day", *relevant_holidays]] = data[["work_day", *relevant_holidays]].fillna(0)

time: 4.63 s (started: 2024-02-22 09:36:11 +00:00)


In [ ]:
# include date-related future covariates
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_year'] = data['date'].dt.dayofyear
data['week_of_year'] = data['date'].dt.isocalendar().week.astype('int32')
data['date_index'] = data['date'].factorize()[0]    # trend

time: 1.1 s (started: 2024-02-22 09:36:16 +00:00)


In [ ]:
# impute days with zero sales using linear interpolation later
# there were no sales on new year's days
missing_dates = pd.date_range(train_start, train_end, freq='D').difference(train['date'].unique())
zero_sales_dates = missing_dates.union(pd.date_range(f"{train_start.year}", f"{test_end.year}", freq='YS'))

data['onpromotion'] = data['onpromotion'].astype('float64')
zero_sales_mask = data['date'].isin(zero_sales_dates) & (data['sales'] == 0) & (data['onpromotion'] == 0)
data.loc[zero_sales_mask, ['sales', 'onpromotion']] = float('nan')

time: 54.3 ms (started: 2024-02-22 09:36:17 +00:00)


## Train model

### Prepare raw training data, and past and future covariates

In [ ]:
past_cols = ["transactions"]

future_cols = [
    "oil", "onpromotion",
    "day", "month", "year", "day_of_week", "day_of_year", "week_of_year", "date_index",
    "work_day", *relevant_holidays
]
future_ma_cols = ["oil", "onpromotion"]
future_window_sizes = [7, 28]

lags = {
    "target": sorted(range(1, 64), reverse=True),
    "past": sorted(range(16, 23), reverse=True),
    "future": [-i for i in range(-14, 1)]
}

time: 994 µs (started: 2024-02-22 09:36:17 +00:00)


### Prepare training data

In [ ]:
predictions = {}
for family, g in data.groupby('family'):

    df, scaler, dummy = get_target_data(g, train_end=train_end)
    past = get_past_covariates(g, past_cols, train_end)
    future = get_future_covariates(g, future_cols, future_ma_cols, future_window_sizes)

    X_train, y_train = get_training_data(df, past, future, dummy)

    # fit a model
    lr = LinearRegression(n_jobs=-1)
    lr.fit(X_train.values, y_train.values)

    prediction = predict(lr, 16, df, past, future, dummy, scaler, zero_forecast=21)
    predictions[family] = prediction

time: 6min 38s (started: 2024-02-16 03:08:51 +00:00)


In [ ]:
predictions = pd.concat(predictions).stack()

time: 10.3 ms (started: 2024-02-16 03:18:40 +00:00)


## Predictions

In [ ]:
ids = test.set_index(['family', 'date', 'store_nbr'])[['id']]
predictions.index.names = ['family', 'date', 'store_nbr']
predictions.name = 'sales'

time: 14.2 ms (started: 2024-02-16 03:18:45 +00:00)


In [ ]:
df = ids.join(predictions).reset_index(drop=True)
df.to_csv('submission_linreg.csv', index=False)

time: 76.5 ms (started: 2024-02-16 03:18:49 +00:00)


## Scratch

## Benchmark

In [ ]:
aa = 'GROCERY I'
bb = 'BEVERAGES'
g = data[data['family']==bb]

time: 310 ms (started: 2024-02-22 09:37:03 +00:00)


In [ ]:
df, scaler, dummy = get_target_data(g, train_end=train_end)

time: 65.5 ms (started: 2024-02-22 09:37:06 +00:00)


In [ ]:
past = get_past_covariates(g, past_cols, train_end)

time: 17.3 ms (started: 2024-02-22 09:37:07 +00:00)


In [ ]:
future = get_future_covariates(g, future_cols, future_ma_cols, future_window_sizes)

time: 70.9 ms (started: 2024-02-22 09:37:08 +00:00)


In [ ]:
X_train, y_train = get_training_data(df, past, future, dummy)

time: 1.81 s (started: 2024-02-22 09:37:09 +00:00)


In [ ]:
lr = LinearRegression(n_jobs=-1)
lr.fit(X_train.values, y_train.values)

LinearRegression(n_jobs=-1)

time: 3.03 s (started: 2024-02-22 09:37:13 +00:00)


In [ ]:
prediction = predict(lr, 16, df, past, future, dummy, scaler, zero_forecast=21)

time: 558 ms (started: 2024-02-22 09:37:30 +00:00)
